<a href="https://colab.research.google.com/github/RonaldTuna/MW_Exercises/blob/main/ETL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download Data from links given in exercise

HOW TO RUN

In [ ]:
#install pyspark
try:
  %pyspark_version 3.x
except Exception:
  pass

!pip install pyspark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 24.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a92a75c12c95d9732fd9f90e3164c3a454e5060f2eb5ec5cbe423ba862b8e6b5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Test Setup")\
        .getOrCreate()

In [ ]:
#download data
!wget -nc "https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv"
!wget -nc "https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv"
!wget -nc "https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv"

--2022-09-02 18:30:49--  https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv
Resolving als-hiring.s3.amazonaws.com (als-hiring.s3.amazonaws.com)... 52.217.36.244
Connecting to als-hiring.s3.amazonaws.com (als-hiring.s3.amazonaws.com)|52.217.36.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146786949 (140M) [text/csv]
Saving to: ‘cons.csv’

cons.csv            100%[===================>] 139.99M  46.8MB/s    in 3.0s    

2022-09-02 18:30:53 (46.8 MB/s) - ‘cons.csv’ saved [146786949/146786949]

--2022-09-02 18:30:53--  https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv
Resolving als-hiring.s3.amazonaws.com (als-hiring.s3.amazonaws.com)... 52.217.66.124
Connecting to als-hiring.s3.amazonaws.com (als-hiring.s3.amazonaws.com)|52.217.66.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219452618 (209M) [text/csv]
Saving to: ‘cons_email.csv’

cons_email.csv      100%[========

In [ ]:
#load data into data frames
cons_df = spark.read.option("header","true").csv('/content/cons.csv')
cons_email_df = spark.read.option("header","true").csv('/content/cons_email.csv')
cons_email_chap_sub_df = spark.read.option("header","true").csv('/content/cons_email_chapter_subscription.csv')

Construct new data frames to satisfy exercises.



In [ ]:
cons_email_df.show()

#construct new data frame "people"

+-------------+-------+------------------+----------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+------------+-------------+------+----+
|cons_email_id|cons_id|cons_email_type_id|is_primary|               email|canonical_local_part|     domain|   double_validation|           create_dt|create_app|create_user|         modified_dt|modified_app|modified_user|status|note|
+-------------+-------+------------------+----------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+------------+-------------+------+----+
|            1| 548198|              3361|         1|xmartinez@vincent...|                null|  gmail.com|                null|Wed, 1994-01-26 2...|      4072|       9954|Sat, 2014-04-19 1...|        1990|         7595|     1|null|
|            2| 491137|              2474|         1|  hmiller@hayne

In [ ]:
cons_df.show()

+-------+------+---------+----------+--------+------+--------------------+------+---------------+--------------------+--------------------+--------------------+----------------+--------+--------------------+------+----------+------------+---------+--------------------------+---------------+--------------------+----------+-----------+--------------------+------------+-------------+------+--------------------+
|cons_id|prefix|firstname|middlename|lastname|suffix|          salutation|gender|       birth_dt|               title|            employer|          occupation|          income|  source|           subsource|userid|  password|is_validated|is_banned|change_password_next_login|consent_type_id|           create_dt|create_app|create_user|         modified_dt|modified_app|modified_user|status|                note|
+-------+------+---------+----------+--------+------+--------------------+------+---------------+--------------------+--------------------+--------------------+----------------

In [ ]:
cons_email_chap_sub_df.show()

+----------------------------------+-------------+----------+-------+--------------------+--------------------+
|cons_email_chapter_subscription_id|cons_email_id|chapter_id|isunsub|            unsub_dt|         modified_dt|
+----------------------------------+-------------+----------+-------+--------------------+--------------------+
|                                 1|       332188|         1|      1|Sat, 1971-06-12 1...|Thu, 1990-06-28 1...|
|                                 2|       536526|         1|      1|Wed, 2006-07-12 0...|Thu, 1979-09-20 0...|
|                                 3|       134711|         1|      1|Tue, 1987-01-06 1...|Sun, 1974-03-03 1...|
|                                 4|       660345|         1|      1|Sat, 2016-08-06 1...|Wed, 1995-09-13 2...|
|                                 5|       184268|         1|      1|Sun, 2000-05-28 0...|Sat, 1983-12-10 0...|
|                                 6|       621735|         1|      1|Wed, 2009-12-09 1...|Sat, 1990-09-0

In [ ]:
#drop duplicate fields before joining
cons_email_df = cons_email_df.drop("create_dt","modified_dt")
cons_email_chap_sub_df = cons_email_chap_sub_df.drop("modified_dt")

#join cons and cons_email on cons_id
people_df = cons_df.join(cons_email_df, cons_df.cons_id ==  cons_email_df.cons_id,"inner")

#join resulting table with cons_email_chap_sub on con_email_id
people_df = people_df.join(cons_email_chap_sub_df, people_df.cons_email_id ==  cons_email_chap_sub_df.cons_email_id,"inner")

#drop all unecessary fields to create "people" data frame
people_df = people_df.drop("cons_id","prefix","firstname","middlename","lastname","suffix","salutation","gender","birth_dt","title","employer","occupation",
                           "income","subsource","userid","password","is_validated","is_banned","change_password_next_login","consent_type_id","create_app",
                           "create_user","modified_app","modified_user","status","note","cons_email_id","cons_email_type_id","is_primary","canonical_local_part",
                           "domain","double_validation","create_app","create_user","modified_app","modified_user","status",
                           "cons_email_chapter_subscription_id","cons_email_id","chapter_id","unsub_dt")

people_df = people_df.select(col("create_dt"),to_date(col("create_dt"),"E yyyy-MM-DD HH:mm:ss.SSSS").alias("created_dt"))

#rename columns to match exercise
#people_df = people_df.withColumnRenamed("isunsub","is_unsub")
#people_df = people_df.withColumnRenamed("source","code")
#people_df = people_df.withColumnRenamed("modified_dt","updated_dt")
#people_df = people_df.withColumnRenamed("create_dt","created_dt")

#reorder to match exercise
#people_df = people_df.select("email","code","is_unsub","created_dt","updated_dt")

#change schema to match exercise
#people_df = people_df.withColumn("created_dt", 
#                                  people_df["created_dt"]
#                                  .cast('date'))
#people_df = people_df.withColumn("updated_dt", 
#                                  people_df["updated_dt"]
#                                  .cast('date'))
#people_df = people_df.withColumn("is_unsub", 
#                                  people_df["is_unsub"]
#                                  .cast('boolean'))

#save "people" as a .csv
#people_df.write.csv("/content/people.csv")

In [ ]:
people_df.show()

SparkUpgradeException: ignored

In [ ]:
#create "acquisition_facts" by retaining created_dt and renaming it "acquisition date" and grouping 
#by date to see how many acquisition were made on that date, and naming that column "acquisitions"
from pyspark.sql.functions import dayofmonth, count

acquisition_facts_df = (people_df.groupBy(dayofmonth("created_dt").alias("acquisition_date")).count("created_dt").alias("acquisitions"))

#save "acquisition" as a .csv
#acquisition_facts_df.write.csv("/content/acquisition.csv")

TypeError: ignored